In [1]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
%pip install py_vollib
import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta
from py_vollib.black_scholes.implied_volatility import implied_volatility


df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 365 #T = time to expiration in years
df = df.drop(["date"], axis=1)
df.head()

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


,Date,Underlying,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190,T
0,2023-08-21,175.84,16.40,13.27,10.30,7.85,5.91,6.81,8.65,10.85,13.35,16.50,0.413699
1,2023-08-22,177.23,17.50,14.10,11.10,8.51,6.34,6.26,8.10,10.10,12.75,15.50,0.410959
2,2023-08-23,181.12,20.20,16.65,13.45,10.55,8.08,5.25,6.71,8.50,10.70,13.33,0.408219
3,2023-08-24,176.38,16.97,13.75,10.80,8.35,6.25,6.85,8.62,10.75,13.35,16.60,0.405479
4,2023-08-25,178.61,18.50,15.15,11.85,9.19,6.90,5.90,7.50,9.50,11.90,14.81,0.402740


The delta measures the rate of change of the security wrt. to the price of the underlying. Thus, the delta of the stock is one.
Vega measures the rate of change of the security wrt. to the volatility of the underlying. Again, the vega of the stock is one. In delta hedging, we short the option and hold a dynamically adjusted amount of the risky asset to eliminate the risk.

In [2]:
r = 0.01 # can be modified
sigma = 0.5 # calculate later, the volatility of the stock

def get_greeks(type, underlying, strike, T):
    # type is "call" or "put"
    val = BSM(kind=type, S0=underlying, K=strike, T=T, r=r, sigma=sigma).value()
    delta = BSM(kind=type, S0=underlying, K=strike, T=T, r=r, sigma=sigma).delta()
    vega = BSM(kind=type, S0=underlying, K=strike, T=T, r=r, sigma=sigma).vega()
    return val, delta, vega

print(get_greeks("call", 100, 95, 0.25)) #testing

(12.527923392521458, 0.633136941899257, 18.825749309685786)


Next, calculate the hedge with 2 portfolios: OP and RE. OP has a long call and RE has a short amount of the underlying asset (here, $AAPL). We want P = OP + RE to be neutral to small changes in the underlying price.

In [3]:
def delta_hedge():
    OP = C_0 # replace with call opt price
    RE = - delta * underlying # replace again
    next = """ we calculate changes in OP and RE and let A_0 = d OP - d RE"""

#next, apply the delta hedge while looping through the dates in the df!


In [7]:
def delta_hedge(call_option_prices):
    r = 0.5076 # 3-Month Treasury Yield
    implied_vola = 0.2 # initial guess ?
    bsm_obj = BSM(kind='call', S0=call_option_prices.iloc[0]['Underlying'], K=call_option_prices.iloc[0]['C190'], T=call_option_prices.iloc[0]['T'], r=r, sigma=implied_vola)
    OP = bsm_obj.value() # Value of the option according to BSM
    RE = - bsm_obj.delta() * call_option_prices.iloc[0]['Underlying'] # replace again
    delta = bsm_obj.delta()
    
    print("Initial values:")
    print("Value of the option according to BSM: ", OP)
    print("Value of the delta of the option according to BSM: ", delta)
    print("Value of the long position according to BSM: ", OP)
    print("Value of the short position according to BSM: ", RE)

    # next = """ we calculate changes in OP and RE and let A_0 = d OP - d RE"""
    # Hedge for 45 days every second day
    for i in range(2, len(call_option_prices)-1, 1):
        new_bsm_obj = BSM(kind='call', S0=call_option_prices['Underlying'][i], K=call_option_prices['C190'][i], T=call_option_prices['T'][i], r=r, sigma=implied_vola)
        delta = n
        new_OP = new_bsm_obj.value()
        new_RE = - new_bsm_obj.delta() * call_option_prices['Underlying'][i]
        d_OP = new_OP - OP
        d_RE = delta*(call_option_prices['Underlying'][i+1] - call_option_prices['Underlying'][i])
        a_0 = d_OP - d_RE

        print('==================================================')
        print("Date:", call_option_prices['Date'][i])
        print("Value of the new option according to BSM: ", new_OP)
        print("Value of the new delta of the option according to BSM: ", new_bsm_obj.delta())
        print("Value of the new long position according to BSM: ", new_OP)
        print("Value of the new short position according to BSM: ", new_RE)
        print("Difference of the long positions: ", OP - new_OP)
        print("Difference of the short positions: ", RE - new_RE)
        print("A_0: ", a_0)
        print('==================================================')
        print('\n')

        OP = new_OP
        RE = new_RE

        # every second day, t2, t4, t6, so on..
        if i % 2 == 0:
            # rehedge() ??
            delta = new_bsm_obj.delta()
            implied_vola = implied_volatility(new_OP, call_option_prices['Underlying'][i], call_option_prices['C190'][i], call_option_prices['T'][i], r, 'c')
            print("Rehedging date:", call_option_prices['Date'][i])
            print("Delta updated, new delta is:", delta)
            print("Volatility updated, new implied volatility is:", implied_vola)
    
#next, apply the delta hedge while looping through the dates in the df!

In [8]:
delta_hedge(df)

Initial values:
Value of the option according to BSM:  171.04941560668675
Value of the delta of the option according to BSM:  1.0
Value of the long position according to BSM:  171.04941560668675
Value of the short position according to BSM:  -175.84
Date: 2023-08-23
Value of the new option according to BSM:  174.55219411050726
Value of the new delta of the option according to BSM:  1.0
Value of the new long position according to BSM:  174.55219411050726
Value of the new short position according to BSM:  -181.12
Difference of the long positions:  -3.502778503820508
Difference of the short positions:  5.280000000000001
A_0:  8.242778503820517


Rehedging date: 2023-08-23
Delta updated, new delta is: 1.0
Volatility updated, new implied volatility is: 0.0
Date: 2023-08-24
Value of the new option according to BSM:  171.29263459761384
Value of the new delta of the option according to BSM:  1.0
Value of the new long position according to BSM:  171.29263459761384
Value of the new short positio

BelowIntrinsicException: The volatility is below the intrinsic value.